In [1]:
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import tensorflow.keras.backend as K
import tensorflow as tf
import matplotlib.pyplot as plt
import albumentations as A
from osgeo import gdal
import numpy as np
import pandas as pd
import random
import keras
import glob
import cv2
import os

'''

images on the borders are clogging up machine
padding isn't working as the self array samples is coming from unpadded 
so all the coordinates are off.

1. pad the height (y) data aswell so the coordinates are the same

'''


"\n\nimages on the borders are clogging up machine\npadding isn't working as the self array samples is coming from unpadded \nso all the coordinates are off.\n\n1. pad the height (y) data aswell so the coordinates are the same\n\n"

In [2]:
class DataGenerator(keras.utils.Sequence):

    
    def __init__(self, batch_size=32, random_state=42, train_test='train', offset=5,
                 height_dir='C:/Users/egnke/PythonCode/MetEireann/Dublin_Height_Data/tiled/',
                 sentinel_2_dir='C:/Users/egnke/PythonCode/MetEireann/Sentienl-2-Data/Processed_Data/interpolation/',
                 sentinel_1_dir='C:/Users/egnke/PythonCode/MetEireann/Sentinel-1-Data/Sentinel-1/Interpolation/Asc/'):
        
        
        self.offset = offset
        self.batch_size = batch_size
        self.random_state = random_state
        
        if train_test == 'train':
            
            self.tiles  = ['X0002_Y0002','X0002_Y0003','X0003_Y0002']
        else:
            
            self.tiles   = ['X0003_Y0003']
        
        self.bands = ['2020-2020_001-365_HL_TSA_SEN2L_NDV_STM.tif','2020-2020_001-365_HL_TSA_VVVHP_BVH_STM.tif',
                      '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM.tif','2020-2020_001-365_HL_TSA_SEN2L_BNR_STM.tif',
                      '2020-2020_001-365_HL_TSA_SEN2L_SW1_STM.tif','2020-2020_001-365_HL_TSA_SEN2L_NDW_STM.tif',
                      '2020-2020_001-365_HL_TSA_SEN2L_TCG_STM.tif']
        
        self.height_dir = height_dir
        self.sentinel_2_dir = sentinel_2_dir
        self.sentinel_1_dir = sentinel_1_dir
        
        
        self.scene_df = self.create_sampling_array()
        print('finshed creating scene df....')
        self.volumes = self.create_img_volumes(self.tiles)
        print('finshed creating image volumes....')
        self.on_epoch_end()
    
    
    
    def on_epoch_end(self):
        '''Updates indexes after each epoch'''
        self.scene_df = self.scene_df.sample(frac = 1)
    
    
    def __len__(self):
        '''Denotes the number of batches per epoch'''
        return int(np.floor(len(self.scene_df) /(self.batch_size*10)))
    
    
    def __getitem__(self, index):
        '''Generate one batch of data'''
        # Generate indexes of the batch
        sample_ = self.scene_df.sample(n=self.batch_size)

        # Generate data
        X, y = self.__data_generation(sample_)

        return X, y
    
    
    def __data_generation(self, sample_df):
        
        X = np.empty((self.batch_size, *(11,11), 7))
        y = np.empty((self.batch_size, 1))
        
        
        X_temp, y_temp = zip(*sample_df.apply(self.extract_x_y_from_volumes, volume_dict=self.volumes, offset=2, axis=1))
        

        X = np.stack(X_temp, axis=0)
        y = np.stack(y_temp, axis=0)
        
        X = X.astype(np.float32)
        y = y.astype(np.float32)

        return X, y
    
    
         
    def create_sampling_array(self):
        
        dfs = []
        for tile in self.tiles:
            
            height = gdal.Open(self.height_dir+tile+'/IE001L1_Dublin_UA2012_DHM_v010.tif')
            height_data = height.GetRasterBand(1).ReadAsArray()
 
            height_data = np.pad(height_data, pad_width=((6, 6), (6, 6)), 
                                 mode='constant',
                                 constant_values=-9999)
            
            
            height_df = self.convert_2d_array_to_dataframe(height_data, 'heights')
            height_df = height_df[height_df['heights'] > 0]
            height_df['tile'] = tile
            
            dfs.append(height_df)
            
        return pd.concat(dfs)
        
        
        
    @staticmethod
    def convert_2d_array_to_dataframe(raster, colname):
    
        '''
        converts a raster to dataframe where each pixel value is a row and the x y coordinates are there

        np.array([[1,2,3],[4,5,6],[7,8,9]]) ->  y,x,value
                                                0,0,1
                                                0,1,2
                                                0,2,3
                                                1,0,4
                                                1,1,5
                                                1,2,6
                                                2,0,7
                                                2,1,8
                                                2,2,9
        '''

        return (pd.DataFrame(raster)
                 .stack()
                 .rename_axis(['y', 'x'])
                 .reset_index(name=colname))
    
    
    
    @staticmethod
    def stack_images_into_volume(list_of_img_rasters):
        '''
        stacks numpy arrays together along z-axis to create
        multichannel image.
        '''
        return np.dstack(list_of_img_rasters)

    
    
    @staticmethod
    def zero_pad_img(img, before_after_axis=((6, 6), (6, 6), (0, 0))):
        '''
        zero pad numpy array default will pad 250x250x3 -> 256x256x3
        each tuple in tuple refers to an axis ((3,3),(3,3),(0,0))
        so if we wanted to convert a 250x250x250x3 -> 256x256x256x3
        we would write ((3,3),(3,3),(3,3),(0,0))
        '''
        
        padded_img = np.pad(img, before_after_axis, 
                            mode='constant', constant_values=-9999)
        return padded_img
    
    
    
    def create_img_volume(self, tile):
        
        band_list = []
        
        for band in self.bands:
            
            # check for sentinel 1 band
            if 'VVVHP_' in band:
                source = gdal.Open(self.sentinel_1_dir + tile + '/' + band)
                band_data = source.GetRasterBand(7).ReadAsArray()
                band_list.append(band_data)
                
            else:
                source = gdal.Open(self.sentinel_2_dir + tile + '/' + band)
                band_data = source.GetRasterBand(7).ReadAsArray()
                band_list.append(band_data)
                
        return self.stack_images_into_volume(band_list)
    
    
    

    def extract_x_y_from_volumes(self, row, volume_dict, offset):
        img_volume = volume_dict[row['tile']]
        x = row['y']
        y = row['x']
        #x = row['x']
        #y = row['y']
        return img_volume[x-self.offset:x+self.offset+1, y-self.offset:y+self.offset+1], row['heights']
    
    
    
    def create_img_volumes(self, tiles):
        
        volumes = {}
        for tile in tiles:
            volume = self.create_img_volume(tile)
            volume = self.zero_pad_img(volume)
            volumes[tile] = volume
            
        return volumes

        

In [3]:
def create_model():

    model = keras.Sequential()

    model.add(keras.layers.Conv2D(filters=6, kernel_size=(3, 3), activation='linear', input_shape=(11, 11, 7)))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.2))
    

    model.add(keras.layers.Flatten())
    model.add(BatchNormalization())
    model.add(tf.keras.layers.Dropout(0.2))

    model.add(keras.layers.Dense(units=1, activation = 'linear'))
    
              
    return model
    
    
def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true))) 


Model = create_model()
Model.compile(optimizer='adam',  run_eagerly=True, loss=tf.keras.losses.MeanAbsoluteError(), metrics=[tf.keras.metrics.RootMeanSquaredError()])
Model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 9, 9, 6)           384       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 6)           0         
_________________________________________________________________
batch_normalization (BatchNo (None, 4, 4, 6)           24        
_________________________________________________________________
dropout (Dropout)            (None, 4, 4, 6)           0         
_________________________________________________________________
flatten (Flatten)            (None, 96)                0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 96)                384       
_________________________________________________________________
dropout_1 (Dropout)          (None, 96)                0

In [4]:
train_generator = DataGenerator(train_test='train')
test_generator  = DataGenerator(train_test='test')

finshed creating scene df....
finshed creating image volumes....
finshed creating scene df....
finshed creating image volumes....


In [5]:
callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)

Model.fit(
            train_generator,
            epochs=100,
            validation_data=test_generator,
            workers=2, use_multiprocessing=False,
            callbacks=[callback]
)


Epoch 1/100
2669/2669 [==============================] - 150s 56ms/step - loss: 3.2414 - root_mean_squared_error: 4.8256 - val_loss: 1.5346 - val_root_mean_squared_error: 2.4698
Epoch 2/100
2669/2669 [==============================] - 136s 51ms/step - loss: 2.0365 - root_mean_squared_error: 3.4954 - val_loss: 1.5459 - val_root_mean_squared_error: 2.7449
Epoch 3/100
2669/2669 [==============================] - 116s 43ms/step - loss: 1.9730 - root_mean_squared_error: 3.3770 - val_loss: 1.5124 - val_root_mean_squared_error: 2.3491
Epoch 4/100
2669/2669 [==============================] - 125s 47ms/step - loss: 1.9757 - root_mean_squared_error: 3.3887 - val_loss: 1.5530 - val_root_mean_squared_error: 2.4867
Epoch 5/100
2669/2669 [==============================] - 118s 44ms/step - loss: 1.9403 - root_mean_squared_error: 3.3285 - val_loss: 1.5048 - val_root_mean_squared_error: 2.4574
Epoch 6/100
2669/2669 [==============================] - 122s 46ms/step - loss: 1.9233 - root_mean_squared_err

In [21]:
important_features = ['2020-2020_001-365_HL_TSA_SEN2L_NDV_STM_B0007_GRD', '2020-2020_001-365_HL_TSA_SEN2L_NDV_STM_B0007_ERO', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0011_OPN', '2020-2020_001-365_HL_TSA_SEN2L_BNR_STM_B0002_CLS', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0003_DIL', '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0003_GRD', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0011_CLS', '2020-2020_001-365_HL_TSA_SEN2L_TCG_STM_B0008_OPN', 
                     '2020-2020_001-365_HL_TSA_SEN2L_BNR_STM_B0004_OPN', '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0001_DIL', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0012_DIL', '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0006_GRD', 
                     '2020-2020_001-365_HL_TSA_SEN2L_SW1_STM_B0002_ERO', '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0007_DIL', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0002_GRD', '2020-2020_001-365_HL_TSA_SEN2L_NDW_STM_B0001_DIL', 
                     '2020-2020_001-365_HL_TSA_SEN2L_BNR_STM_B0005_ERO', '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0005_GRD', 
                     '2020-2020_001-365_HL_TSA_SEN2L_GRN_STM_B0004_ERO', '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0002_DIL', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0012_DIL', '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0001_DIL', 
                     '2020-2020_001-365_HL_TSA_SEN2L_SW1_STM_B0012_OPN', '2020-2020_001-365_HL_TSA_SEN2L_NIR_STM_B0003_ERO', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0003_ERO', '2020-2020_001-365_HL_TSA_SEN2L_TCW_STM_B0012_ERO', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0005_DIL', '2020-2020_001-365_HL_TSA_SEN2L_NIR_STM_B0007_ERO', 
                     '2020-2020_001-365_HL_TSA_SEN2L_BNR_STM_B0004_ERO', '2020-2020_001-365_HL_TSA_SEN2L_TCG_STM_B0006_ERO', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0005_OPN', '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0007_DIL', 
                     '2020-2020_001-365_HL_TSA_SEN2L_NDB_STM_B0007_DIL', '2020-2020_001-365_HL_TSA_SEN2L_NIR_STM_B0004_ERO', 
                     '2020-2020_001-365_HL_TSA_SEN2L_NDW_STM_B0013_ERO', '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0001_CLS', 
                     '2020-2020_001-365_HL_TSA_SEN2L_GRN_STM_B0004_GRD', '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0001_OPN', 
                     '2020-2020_001-365_HL_TSA_SEN2L_NDW_STM_B0004_DIL', '2020-2020_001-365_HL_TSA_SEN2L_NIR_STM_B0012_OPN', 
                     '2020-2020_001-365_HL_TSA_SEN2L_SW2_STM_B0002_ERO', '2020-2020_001-365_HL_TSA_SEN2L_TCB_STM_B0009_DIL', 
                     '2020-2020_001-365_HL_TSA_SEN2L_NIR_STM_B0002_ERO', '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0006_CLS', 
                     '2020-2020_001-365_HL_TSA_SEN2L_NDW_STM_B0013_OPN', '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0003_DIL', 
                     '2020-2020_001-365_HL_TSA_SEN2L_TCW_STM_B0007_GRD', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0012_CLS', '2020-2020_001-365_HL_TSA_SEN2L_RE2_STM_B0011_GRD', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0004_GRD', '2020-2020_001-365_HL_TSA_SEN2L_TCB_STM_B0012_CLS', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0008_GRD', '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0012_GRD', 
                     '2020-2020_001-365_HL_TSA_SEN2L_RE1_STM_B0001_CLS', '2020-2020_001-365_HL_TSA_SEN2L_TCG_STM_B0003_OPN', 
                     '2020-2020_001-365_HL_TSA_SEN2L_TCG_STM_B0003_BHT', '2020-2020_001-365_HL_TSA_SEN2L_SW1_STM_B0003_ERO', 
                     '2020-2020_001-365_HL_TSA_SEN2L_NIR_STM_B0002_CLS', '2020-2020_001-365_HL_TSA_SEN2L_NDB_STM_B0011_CLS', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0009_ERO', '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0007_CLS', 
                     '2020-2020_001-365_HL_TSA_SEN2L_SW1_STM_B0007_ERO', '2020-2020_001-365_HL_TSA_SEN2L_TCB_STM_B0013_ERO', 
                     '2020-2020_001-365_HL_TSA_SEN2L_SW1_STM_B0004_ERO', '2020-2020_001-365_HL_TSA_SEN2L_RE1_STM_B0011_ERO', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0001_GRD', '2020-2020_001-365_HL_TSA_SEN2L_RE1_STM_B0011_OPN', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0001_CLS', '2020-2020_001-365_HL_TSA_SEN2L_TCB_STM_B0004_ERO', 
                     '2020-2020_001-365_HL_TSA_SEN2L_NIR_STM_B0001_ERO', '2020-2020_001-365_HL_TSA_VVVHP_BVV_STM_B0007_OPN', 
                     '2020-2020_001-365_HL_TSA_VVVHP_BVH_STM_B0007_GRD', '2020-2020_001-365_HL_TSA_SEN2L_RE2_STM_B0008_ERO']


len(important_features)

73

In [ ]:
def start_points(size, chunk_size, overlap=0):
    '''
    split an array into equal parts including overlap. used
    as the starting points then we add the size to each point to get
    the desired result.
    
    chunk_size: is the size of the stride length to take 
                i.e chunk_size = 2 [0,1,2,3] -> [0,2]
    overlap: is a float 0.5=50% of the overlap to take between chunk_sizes
            chunk_size:4, np.arange(10), overlap:0.5 [0, 4, 6] -> [0, 2, 4, 6]
    '''
    
    if(overlap>1 or overlap<0): raise ValueError('The Overlap Parameter must be between 0-1!')
    
    points = [0]
    stride = int(chunk_size * (1-overlap))
    counter = 1
    while True:
        
        pt = stride * counter
        
        if pt + chunk_size >= size:
            points.append(size - chunk_size)
            break
        else:
            points.append(pt)
        
        counter += 1
        
    return points


chunk_size = 2
x = np.array([0,1,2,3])
start_points(len(x), chunk_size, overlap=0)